In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from reservoirpy.nodes import Reservoir, Ridge, ESN
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import datetime

In [2]:
# # Chemin vers la base de données SQLite
# storage_url = "sqlite:///db.sqlite3"

# # Charger l'étude existante
# study = optuna.load_study(
#     study_name="no-name-e82af600-5504-486b-95cb-12b9e9a1ded9",  # Remplacer par le nom de votre étude
#     storage=storage_url
# )

# # Récupérer les meilleurs paramètres
# best_params = study.best_params
# print("Meilleurs paramètres :", best_params)

In [3]:
# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
# EMBEDDING_SIZE = 256  # Taille des embeddings pour tous les inputs
# NUM_HEADS = 8
# NUM_LAYERS = 4
# DROPOUT = 0.1
# LEARNING_RATE = 1e-4
# EPOCHS = 20
# PATIENCE = 5

In [4]:
class MultiTaskTransformer(nn.Module):
    def __init__(
        self, input_size, embedding_size, num_heads, num_layers, num_labels, dropout
    ):
        super(MultiTaskTransformer, self).__init__()
        self.embedding = nn.Linear(input_size, embedding_size)  # Embedding Layer
        self.transformer = nn.Transformer(
            d_model=embedding_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dropout=dropout,
            batch_first=True,
        )
        self.classifier = nn.Linear(embedding_size, num_labels)  # Final Classifier

    def forward(self, x):
        # Pass input through the embedding layer
        embedded = self.embedding(x)

        # Add a positional encoding (if needed)
        embedded = embedded.unsqueeze(1)  # Add sequence dimension

        # Transformer expects (batch, seq_len, embedding_size)
        transformer_output = self.transformer(embedded, embedded)

        # Take only the output of the first token (classification token equivalent)
        output = transformer_output[:, 0, :]  # Extract first token

        # Pass through the classifier
        predictions = self.classifier(output)
        return predictions


In [5]:
def load_data():
    partOfData = 1
    X_genres = pd.read_csv("../data/test/input_genres_tags_data.csv")
    X_instruments = pd.read_csv("../data/test/input_instruments_tags_data.csv")
    X_moods = pd.read_csv("../data/test/input_moods_tags_data.csv")
    X_genres_categories = pd.read_csv("../data/test/input_genres_categories_data.csv")
    X_instruments_categories = pd.read_csv(
        "../data/test/input_instruments_categories_data.csv"
    )
    X_moods_categories = pd.read_csv("../data/test/input_moods_categories_data.csv")

    y_genres = pd.read_csv("../data/test/output_genres_tags_data.csv")
    y_instruments = pd.read_csv("../data/test/output_instruments_tags_data.csv")
    y_moods = pd.read_csv("../data/test/output_moods_tags_data.csv")

    # On peut garder seulement une partie des données
    X_genres = X_genres[: int(partOfData * len(X_genres))]
    X_instruments = X_instruments[: int(partOfData * len(X_instruments))]
    X_moods = X_moods[: int(partOfData * len(X_moods))]
    X_genres_categories = X_genres_categories[
        : int(partOfData * len(X_genres_categories))
    ]
    X_instruments_categories = X_instruments_categories[
        : int(partOfData * len(X_instruments_categories))
    ]
    X_moods_categories = X_moods_categories[: int(partOfData * len(X_moods_categories))]
    y_genres = y_genres[: int(partOfData * len(y_genres))]
    y_instruments = y_instruments[: int(partOfData * len(y_instruments))]
    y_moods = y_moods[: int(partOfData * len(y_moods))]

    return (
        X_genres,
        X_instruments,
        X_moods,
        X_genres_categories,
        X_instruments_categories,
        X_moods_categories,
        y_genres,
        y_instruments,
        y_moods,
    )


# Ensure the input data is in the correct format
def reshape_input(X):
    if isinstance(X, pd.DataFrame):
        return X.values.reshape(-1, 1, X.shape[1])  # Handles pandas DataFrame
    elif isinstance(X, np.ndarray):
        return X.reshape(-1, 1, X.shape[1])  # Handles numpy ndarray
    else:
        raise ValueError("Input must be a pandas DataFrame or a numpy ndarray")


def format_predictions(predictions):
    # Convert the list to a NumPy array
    predictions_array = np.array(predictions)

    # Reshape the array to 2-dimensional
    predictions_reshaped = predictions_array.reshape(-1, predictions_array.shape[-1])

    return predictions_reshaped

In [6]:
# Charger les données
(
    X_genres,
    X_instruments,
    X_moods,
    X_genres_categories,
    X_instruments_categories,
    X_moods_categories,
    y_genres,
    y_instruments,
    y_moods,
) = load_data()

# Préparation des données
X_genres = X_genres.drop(columns=["ChallengeID"])
X_instruments = X_instruments.drop(columns=["ChallengeID"])
X_moods = X_moods.drop(columns=["ChallengeID"])
X_genres_categories = X_genres_categories.drop(columns=["ChallengeID"])
X_instruments_categories = X_instruments_categories.drop(columns=["ChallengeID"])
X_moods_categories = X_moods_categories.drop(columns=["ChallengeID"])
y_genres = y_genres.drop(columns=["ChallengeID"])
y_instruments = y_instruments.drop(columns=["ChallengeID"])
y_moods = y_moods.drop(columns=["ChallengeID"])


X = np.concatenate(
    [
        X_genres,
        X_instruments,
        X_moods,
        X_genres_categories,
        X_instruments_categories,
        X_moods_categories,
    ],
    axis=1,
)

y = np.concatenate([y_genres, y_instruments, y_moods], axis=1)

# Convertir les données en tensors PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(DEVICE)

y_tensor = torch.tensor(y, dtype=torch.float32).to(DEVICE)

In [7]:
# Load ENSs models
sub_folder = 0
while os.path.exists(f"../models/model_{sub_folder}"):
    sub_folder += 1
sub_folder -= 1

print(f"Loading ESNs models from ../models/model_{sub_folder}")

with open(f"../models/model_{sub_folder}/esn_Genre.pkl", "rb") as f:
    model_Genre = pickle.load(f)

with open(f"../models/model_{sub_folder}/esn_Instrument.pkl", "rb") as f:
    model_Instrument = pickle.load(f)

with open(f"../models/model_{sub_folder}/esn_Mood.pkl", "rb") as f:
    model_Mood = pickle.load(f)

# Load Transformer model
with open(f"../models/model_{sub_folder}/transformer.pkl", "rb") as f:
    model_Transformer = pickle.load(f)

# Affichage du nombre de paramètres
print(
    f"Nombre total de paramètres : {sum(p.numel() for p in model_Transformer.parameters())}"
)

Loading ESNs models from ../models/model_5
Nombre total de paramètres : 6866110


In [8]:
X_genres_reshaped = reshape_input(X_genres)
X_instruments_reshaped = reshape_input(X_instruments)
X_moods_reshaped = reshape_input(X_moods)

y_genres_reshaped = reshape_input(y_genres)
y_instruments_reshaped = reshape_input(y_instruments)
y_moods_reshaped = reshape_input(y_moods)

In [9]:
# Obtenir les sorties des réservoirs
y_genres_pred = model_Genre.run(X_genres_reshaped)
y_instruments_pred = model_Instrument.run(X_instruments_reshaped)
y_moods_pred = model_Mood.run(X_moods_reshaped)

# Formater les prédictions
y_genres_pred = format_predictions(y_genres_pred)
y_instruments_pred = format_predictions(y_instruments_pred)
y_moods_pred = format_predictions(y_moods_pred)


# Combine toutes les sorties
X_final = np.concatenate(
    [
        X_genres,
        X_instruments,
        X_moods,
        y_genres_pred,
        y_instruments_pred,
        y_moods_pred,
        X_genres_categories,
        X_instruments_categories,
        X_moods_categories,
    ],
    axis=1,
)

Running ESN-2: 100%|██████████| 47615/47615 [00:01<00:00, 34502.20it/s]


In [10]:
# Création des datasets
main_dataset = torch.utils.data.TensorDataset(
    torch.tensor(X_final, dtype=torch.float32).to(DEVICE),
    y_tensor.clone().detach().to(DEVICE),
)

# Création des loaders
main_loader = torch.utils.data.DataLoader(
    main_dataset, batch_size=BATCH_SIZE, shuffle=True
)

In [24]:
# Evaluation des performances (accuracy, precision, recall, f1-score)
def evaluate_performance(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            predictions = model(X_batch)
            predictions = torch.sigmoid(predictions)
            predictions = (predictions > 0.5).int()
            y_true.append(y_batch.cpu().numpy())
            y_pred.append(predictions.cpu().numpy())
    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    # Save 5% of the rows of the predictions as csv files in the data folder in predictions folder with timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    np.savetxt(
        f"../data/predictions/test/y_true_{timestamp}.csv",
        y_true[: int(0.05 * len(y_true))],
        delimiter=",",
    )
    np.savetxt(
        f"../data/predictions/test/y_pred_{timestamp}.csv",
        y_pred[: int(0.05 * len(y_pred))],
        delimiter=",",
    )

    # Histograms plot of the predictions and true values for each tag
    # for i in range(y_true.shape[1]):
    #     plt.hist(y_true[:, i], bins=2, alpha=0.5, label="Benchmark")
    #     plt.hist(y_pred[:, i], bins=2, alpha=0.5, label="Predicted")
    #     plt.title(f"Tag {i}")
    #     plt.legend()
    #     plt.savefig(f"../data/predictions/test/plots/histogram_tag_{i}.png")
    #     plt.clf()

    y_true_total = np.sum(y_true, axis=1)
    y_pred_total = np.sum(y_pred, axis=1)

    plt.hist(y_true_total, bins=20, alpha=0.5, label="Benchmark")
    plt.hist(y_pred_total, bins=20, alpha=0.5, label="Predicted")
    plt.title("Total number of tags")
    plt.xlabel("Number of tags")
    plt.ylabel("Frequency")
    plt.legend()
    plt.savefig("../data/predictions/test/plots/histogram_total_tags.png")
    plt.clf()

    # Accuracy
    accuracy = np.mean(y_true == y_pred)
    print(f"Accuracy: {accuracy}")

    # Precision, Recall, F1-Score
    from sklearn.metrics import classification_report

    report = classification_report(y_true, y_pred, digits=4)
    print(report)


In [25]:
# Evaluation du modèle
evaluate_performance(model_Transformer, main_loader)

Accuracy: 0.9912676609769895
              precision    recall  f1-score   support

           0     0.0159    0.5000    0.0308         2
           1     0.7452    0.7074    0.7258      1261
           2     0.0000    0.0000    0.0000         0
           3     0.0000    0.0000    0.0000         0
           4     0.1028    1.0000    0.1864        11
           5     0.1879    0.8689    0.3090        61
           6     0.1130    1.0000    0.2031        13
           7     0.1375    0.9024    0.2387        41
           8     0.4425    0.8597    0.5843       506
           9     0.0984    1.0000    0.1791        24
          10     0.8276    0.9421    0.8811      1243
          11     0.8571    0.8774    0.8672      4226
          12     0.3350    0.9273    0.4922       220
          13     0.1816    0.8191    0.2973       188
          14     0.1575    0.8833    0.2673       120
          15     0.2273    0.9167    0.3642        60
          16     0.0000    0.0000    0.0000         

/home/elouarn/Documents/Projet_IA/challenge-data-music-catalogs/music-catalogs-classifier/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/elouarn/Documents/Projet_IA/challenge-data-music-catalogs/music-catalogs-classifier/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/elouarn/Documents/Projet_IA/challenge-data-music-catalogs/music-catalogs-classifier/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill

<Figure size 640x480 with 0 Axes>

In [26]:
# Load the tag correspondences
tag_correspondences = pd.read_csv("../data/mewo-labels.csv")

# Load latest y_true and y_pred from test predictions folder
# Get the latest timestamped files
predictions_folder = "../data/predictions/test/"
y_true_files = sorted(
    [f for f in os.listdir(predictions_folder) if f.startswith("y_true_")]
)
y_pred_files = sorted(
    [f for f in os.listdir(predictions_folder) if f.startswith("y_pred_")]
)

latest_y_true_file = y_true_files[-1]
latest_y_pred_file = y_pred_files[-1]

# Load the latest y_true and y_pred
y_true = np.loadtxt(os.path.join(predictions_folder, latest_y_true_file), delimiter=",")
y_pred = np.loadtxt(os.path.join(predictions_folder, latest_y_pred_file), delimiter=",")

# Randomly select 20 songs
num_songs = 20
random_indices = np.random.choice(len(y_true), num_songs, replace=False)


# Extract the true and predicted tags for these songs
true_tags = y_true[random_indices]
predicted_tags = y_pred[random_indices]

# Map the tags to their names
tag_names = tag_correspondences.columns[1:]  # Skip the first column which is not a tag

# Display the true and predicted tags
for i in range(num_songs):
    true_tag_names = [
        tag_names[j] for j in range(len(tag_names)) if true_tags[i][j] == 1
    ]
    predicted_tag_names = [
        tag_names[j] for j in range(len(tag_names)) if predicted_tags[i][j] == 1
    ]
    print(f"Song {i + 1}:")
    print(f"  True Tags: {', '.join(true_tag_names)}")
    print(f"  Predicted Tags: {', '.join(predicted_tag_names)}")
    print()

Song 1:
  True Tags: drum-machine, electric-guitar, synthesizer, confident, driving
  Predicted Tags: bass-guitar, drum-machine, electric-guitar, synthesizer, confident, driving, exciting, spectacular

Song 2:
  True Tags: 
  Predicted Tags: world

Song 3:
  True Tags: acoustic-guitar
  Predicted Tags: lounge, acoustic-guitar, carefree, relaxed

Song 4:
  True Tags: vintage-jazz, male-vocals, vocal
  Predicted Tags: vintage-jazz, acoustic-guitar, vocal, romantic, sad

Song 5:
  True Tags: strings-section, relaxed
  Predicted Tags: strings-section, synth-pad, synthesizer, relaxed

Song 6:
  True Tags: sinister
  Predicted Tags: vocal, sinister, suspenseful

Song 7:
  True Tags: acoustic-guitar, relaxed, romantic
  Predicted Tags: vintage-jazz, acoustic-guitar, relaxed, romantic

Song 8:
  True Tags: drum-kit, piano, strings-section
  Predicted Tags: piano, strings-section, hopeful, reflective, sad

Song 9:
  True Tags: contemporary-classical, symphony-orchestra, confident, epic, majesti